In [1]:
!pip install -q gptqmodel==1.7.4 datasets

In [2]:
import torch
import gc
from gptqmodel import GPTQModel, QuantizeConfig
from datasets import load_dataset
import os

In [3]:
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

In [4]:
gc.collect()
torch.cuda.empty_cache()

In [5]:
MODEL_ID = "microsoft/phi-2"
QUANT_OUTPUT_DIR = "./microsoft/phi-2-gptq-4bit"

In [6]:
BITS = 4
GROUP_SIZE = 128
BATCH_SIZE = 1
NUM_SAMPLES = 256

In [7]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
calibration_data = [t for t in dataset["text"] if len(t) > 200][:NUM_SAMPLES]

In [8]:
len(calibration_data)

256

In [9]:
quantize_config = QuantizeConfig(
    bits=BITS,
    group_size=GROUP_SIZE,
    desc_act=False,  # Disable for memory saving
    sym=True,
    true_sequential=True,
    damp_percent=0.01,
)

In [10]:
gc.collect()
torch.cuda.empty_cache()

In [11]:
model = GPTQModel.load(
    MODEL_ID,
    quantize_config=quantize_config,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
)

INFO - Estimated Quantization BPW (bits per weight): 4.1875 bpw, based on [bits: 4, group_size: 128]


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
gc.collect()
torch.cuda.empty_cache()

In [13]:
import time
start_time = time.time()

In [14]:
model.quantize(
    calibration_data,
    batch_size=BATCH_SIZE,
)

INFO - Auto pick kernel based on compatibility: <class 'gptqmodel.nn_modules.qlinear.exllama.ExllamaQuantLinear'>
WARNING - The average length of input_ids of calibration_dataset should be greater than 256: actual avg: 166.828125.


  |█---------------------------------------| 0:00:00 / 0:00:00 [1/32] 3.1%

INFO - {'layer': 0, 'module': 'self_attn.q_proj', 'loss': '2.11823', 'damp': '0.01000', 'time': '2.918', 'fwd_time': '2.184'}
INFO - {'layer': 0, 'module': 'self_attn.k_proj', 'loss': '2.69769', 'damp': '0.01000', 'time': '1.671', 'fwd_time': '1.196'}
INFO - {'layer': 0, 'module': 'self_attn.v_proj', 'loss': '0.74253', 'damp': '0.01000', 'time': '1.669', 'fwd_time': '0.882'}
INFO - {'layer': 0, 'module': 'self_attn.dense', 'loss': '0.41192', 'damp': '0.01000', 'time': '1.785', 'fwd_time': '2.578'}
INFO - {'layer': 0, 'module': 'mlp.fc1', 'loss': '2.89263', 'damp': '0.01000', 'time': '1.269', 'fwd_time': '0.997'}
INFO - {'layer': 0, 'module': 'mlp.fc2', 'loss': '0.17488', 'damp': '0.01000', 'time': '5.149', 'fwd_time': '4.618'}


 Quantizing mlp.fc2 in layer 0 of 31 |██--------------------------------------| 0:00:32 / 0:08:32 [2/32] 6.2%

INFO - {'layer': 1, 'module': 'self_attn.q_proj', 'loss': '2.02360', 'damp': '0.01000', 'time': '1.021', 'fwd_time': '0.777'}
INFO - {'layer': 1, 'module': 'self_attn.k_proj', 'loss': '2.12476', 'damp': '0.01000', 'time': '1.016', 'fwd_time': '0.777'}
INFO - {'layer': 1, 'module': 'self_attn.v_proj', 'loss': '0.51589', 'damp': '0.01000', 'time': '1.366', 'fwd_time': '0.777'}
INFO - {'layer': 1, 'module': 'self_attn.dense', 'loss': '0.21705', 'damp': '0.01000', 'time': '0.968', 'fwd_time': '0.784'}
INFO - {'layer': 1, 'module': 'mlp.fc1', 'loss': '6.51464', 'damp': '0.01000', 'time': '1.146', 'fwd_time': '0.788'}
INFO - {'layer': 1, 'module': 'mlp.fc2', 'loss': '0.43496', 'damp': '0.01000', 'time': '5.226', 'fwd_time': '4.663'}


...

INFO - {'layer': 31, 'module': 'self_attn.q_proj', 'loss': '157.42490', 'damp': '0.01000', 'time': '1.020', 'fwd_time': '0.784'}
INFO - {'layer': 31, 'module': 'self_attn.k_proj', 'loss': '28.94958', 'damp': '0.01000', 'time': '1.033', 'fwd_time': '0.789'}
INFO - {'layer': 31, 'module': 'self_attn.v_proj', 'loss': '1.76336', 'damp': '0.01000', 'time': '0.990', 'fwd_time': '0.782'}
INFO - {'layer': 31, 'module': 'self_attn.dense', 'loss': '0.59219', 'damp': '0.01000', 'time': '1.440', 'fwd_time': '0.828'}
INFO - {'layer': 31, 'module': 'mlp.fc1', 'loss': '18.25520', 'damp': '0.01000', 'time': '1.243', 'fwd_time': '0.795'}
INFO - {'layer': 31, 'module': 'mlp.fc2', 'loss': '3.60578', 'damp': '0.01000', 'time': '5.695', 'fwd_time': '4.707'}
INFO - Quantization summary:
[{'layer': 0, 'module': 'self_attn.q_proj', 'loss': '2.11823', 'damp': '0.01000', 'time': '2.918', 'fwd_time': '2.184'}, {'layer': 0, 'module': 'self_attn.k_proj', 'loss': '2.69769', 'damp': '0.01000', 'time': '1.671', 'fwd_

 Packing model.layers.31.mlp.fc2 |----------------------------------------| 100.0%

INFO - Model packed.


 Quantizing mlp.fc2 in layer 31 of 31 |----------------------------------------| 100.0%

[{'layer': 0,
  'module': 'self_attn.q_proj',
  'loss': '2.11823',
  'damp': '0.01000',
  'time': '2.918',
  'fwd_time': '2.184'},
 {'layer': 0,
  'module': 'self_attn.k_proj',
  'loss': '2.69769',
  'damp': '0.01000',
  'time': '1.671',
  'fwd_time': '1.196'},
 {'layer': 0,
  'module': 'self_attn.v_proj',
  'loss': '0.74253',
  'damp': '0.01000',
  'time': '1.669',
  'fwd_time': '0.882'},
 {'layer': 0,
  'module': 'self_attn.dense',
  'loss': '0.41192',
  'damp': '0.01000',
  'time': '1.785',
  'fwd_time': '2.578'},
 {'layer': 0,
  'module': 'mlp.fc1',
  'loss': '2.89263',
  'damp': '0.01000',
  'time': '1.269',
  'fwd_time': '0.997'},
 {'layer': 0,
  'module': 'mlp.fc2',
  'loss': '0.17488',
  'damp': '0.01000',
  'time': '5.149',
  'fwd_time': '4.618'},
 {'layer': 1,
  'module': 'self_attn.q_proj',
  'loss': '2.02360',
  'damp': '0.01000',
  'time': '1.021',
  'fwd_time': '0.777'},
 {'layer': 1,
  'module': 'self_attn.k_proj',
  'loss': '2.12476',
  'damp': '0.01000',
  'time': '1.0

In [15]:
elapsed = time.time() - start_time

In [25]:
print(f"✅ QUANTIZATION COMPLETE in {elapsed/60:.1f} minutes")

✅ QUANTIZATION COMPLETE in 14.8 minutes


In [16]:
model.save(QUANT_OUTPUT_DIR)

files = os.listdir(QUANT_OUTPUT_DIR)
total_size = sum(
    os.path.getsize(os.path.join(QUANT_OUTPUT_DIR, f))
    for f in files
    if os.path.isfile(os.path.join(QUANT_OUTPUT_DIR, f))
) / (1024**2)

print(f"✅ Saved: {QUANT_OUTPUT_DIR} ({total_size:.1f} MB)")

INFO - Pre-Quantized model size: 5301.87MB, 5.18GB
INFO - Quantized model size: 1751.62MB, 1.71GB
INFO - Size difference: 3550.25MB, 3.47GB - 66.96%


✅ Saved: ./microsoft/phi-2-gptq-4bit (1756.2 MB)


In [17]:
gc.collect()
torch.cuda.empty_cache()

In [18]:
model = GPTQModel.load(QUANT_OUTPUT_DIR, device="cuda:0")

INFO - Estimated Quantization BPW (bits per weight): 4.1875 bpw, based on [bits: 4, group_size: 128]
`torch_dtype` is deprecated! Use `dtype` instead!
INFO - Converting `checkpoint_format` from `gptq` to `gptq_v2`.
INFO - Conversion complete: 0.18242835998535156s


In [20]:
prompt = "What is artificial intelligence?"
result = model.generate(prompt, max_new_tokens=250)[0]
output = model.tokenizer.decode(result, skip_special_tokens=True)

print(f"\nPrompt: {prompt}")
print(f"Output: {output}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Prompt: What is artificial intelligence?
Output: What is artificial intelligence?
AI is the ability of machines to perform tasks that normally require human intelligence, such as learning, reasoning, and problem-solving. AI can be classified into two types: narrow AI and general AI. Narrow AI is the type of AI that can perform specific tasks, such as playing chess, recognizing faces, or driving cars. General AI is the type of AI that can perform any task that a human can, but it is still under development and not yet achieved.

What are the benefits of artificial intelligence?
AI has many benefits for various fields and industries, such as:

- Improving the quality and efficiency of products and services
- Enhancing the accuracy and speed of decision-making
- Reducing the costs and risks of human errors
- Creating new opportunities and innovations
- Solving complex and global problems

What are the challenges of artificial intelligence?
AI also has some challenges and risks, such as:
